<a href="https://colab.research.google.com/github/ggnanaraj/Story/blob/main/Stock0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install nselib

In [3]:
pip install numpy

In [ ]:
import numpy as np

In [ ]:
import nselib
#data = nselib.trading_holiday_calendar()
#print(data)

In [ ]:
from nselib import capital_market

In [ ]:
data=capital_market.bhav_copy_with_delivery("01-03-2024")
print(data.columns)

Index(['SYMBOL', 'SERIES', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE',
       'CLOSE_PRICE', 'PREV_CLOSE', 'TTL_TRD_QNTY', 'TURNOVER_LACS',
       'NO_OF_TRADES', 'DELIV_QTY', 'DELIV_PER', 'DATE1'],
      dtype='object')


In [ ]:
#data1=data('SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE','TTL_TRD_QNTY','DELIV_PER')
data1=data.loc[1:,['SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE','TTL_TRD_QNTY','DELIV_PER']]
data1['GAP_UP']=(((data1['OPEN_PRICE']-data1['PREV_CLOSE'])/data1['PREV_CLOSE'])*100).round(2)


In [4]:
import nselib
from nselib import capital_market
data=capital_market.bhav_copy_with_delivery("07-03-2024")
data1=data.loc[1:,['SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE']]
data1['GAP_UP']=(((data1['OPEN_PRICE']-data1['PREV_CLOSE'])/data1['PREV_CLOSE'])*100).round(2)
data1['TTL_TRD_QNTY']=0
data1['DELIV_PER']=0

#data1=data1[data1['DELIV_PER'] != "-"]
data_5PC_GAP_UP=data1[data1['GAP_UP'] > 5]     # gap greater than 5
data_5PC_GAP_UP=data_5PC_GAP_UP.sort_values(by=['GAP_UP'],ascending=False)
data_PC=data_5PC_GAP_UP[data_5PC_GAP_UP['PREV_CLOSE'] > 10] # only share value above Rs10
data_PC.drop_duplicates(subset='SYMBOL',keep='first',inplace=True) # drop duplicate
data_PC=data_PC.sort_values(by=['SYMBOL'])
list_SYM=data_PC['SYMBOL'].to_list()
list_SYM.sort()
#---------------------------------
dataO=capital_market.bhav_copy_with_delivery("06-03-2024")
data1O=dataO.loc[1:,['SYMBOL','TTL_TRD_QNTY','DELIV_PER']]
data1O=data1O[data1O['DELIV_PER'] != "-"]
data2O=data1O[data1O['SYMBOL'].isin(list_SYM)]# list_SYM or data_PC or list_PC
data2O.drop_duplicates(subset='SYMBOL',keep='first',inplace=True) # drop duplicate
data2O=data2O.sort_values(by=['SYMBOL'])
listO_SYM=data2O['SYMBOL'].to_list()
listO_SYM.sort()
#---------------------------
list_comn=[symbol for symbol in list_SYM if symbol in listO_SYM]
list_diff=[symbol for symbol in list_SYM if symbol not in listO_SYM]
list_indexDel=[]
data_PC1= data_PC
for i in list_diff:
  list_1=data_PC.index[data_PC['SYMBOL'] == i ].to_list()
  data_PC1.drop(list_1, inplace=True) # drops row which is not common
  list_indexDel.extend(list_1)
for j,k in enumerate(list_comn):
  PC1=data2O.loc[data2O['SYMBOL']==list_comn[j],'TTL_TRD_QNTY']
  data_PC1.loc[data_PC1['SYMBOL']==list_comn[j],'TTL_TRD_QNTY']= int(PC1)
  PC1=data2O.loc[data2O['SYMBOL']==list_comn[j],'DELIV_PER']
  data_PC1.loc[data_PC1['SYMBOL']==list_comn[j],'DELIV_PER']= float(PC1)

#data2 = capital_market.price_volume_and_deliverable_position_data(symbol='SBIN', from_date='29-02-2024', to_date='01-03-2024')
#data3=capital_market.equity_list( )
data_VOL=data_PC1[data_PC1['TTL_TRD_QNTY'] > 50000] #Quantity greater than 50000
data_VOL=data_VOL[data_VOL['DELIV_PER'] != "-"]
#data_VOL=data_VOL[data_VOL['DELIV_PER'] > 20]  #
#data_VOL=data_VOL[data_VOL['DELIV_PER'] < 90]
list_N=data_VOL['SYMBOL'].to_list()
data_VOL['GAP_down']=(((data_VOL['HIGH_PRICE']-data_VOL['LOW_PRICE'])/data_VOL['HIGH_PRICE'])*100).round(2)
data_VOL['INT_Hike']=(((data_VOL['HIGH_PRICE']-data_VOL['OPEN_PRICE'])/data_VOL['OPEN_PRICE'])*100).round(2)
#data_VOL


<ipython-input-4-96838e231a93>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_PC.drop_duplicates(subset='SYMBOL',keep='first',inplace=True) # drop duplicate
<ipython-input-4-96838e231a93>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2O.drop_duplicates(subset='SYMBOL',keep='first',inplace=True) # drop duplicate


In [44]:
#print(len(data_PC1))
#print(len(data2O))
import pandas as pd

data_VOL1=[]
Hi_Max1=[]
Hi_Min1=[]
Hi_Ave1=[]
Lo_Max1=[]
Lo_Min1=[]
Lo_Ave1=[]
for i in range(len(list_N)):
  data_T=capital_market.price_volume_data(list_N[i],period='1M').tail(14)
  #print(data_T)
  #-------------------------------------------------
  data_Ta=data_T
  data_Tr=data_T[:-1]
  data_Tr= data_Tr.tail(7)
  Hi_Max=data_Tr['HighPrice'].max()
  Hi_Min=data_Tr['HighPrice'].min()

  data_Hi=data_Tr['HighPrice'].astype('string')
  for j in range(len(data_Hi)):
    data_Hi.iloc[j]=data_Hi.iloc[j].replace(",","")
  data_Hi=data_Hi.astype('string').astype(float)
  #print(data_T)
  Hi_Ave=data_Hi.mean(skipna=True).astype(int)


  Lo_Max=data_Tr['LowPrice'].max()
  Lo_Min=data_Tr['LowPrice'].min()

  data_Lo=data_Tr['LowPrice'].astype('string')
  for j in range(len(data_Lo)):
    data_Lo.iloc[j]=data_Lo.iloc[j].replace(",","")
  data_Lo=data_Lo.astype('string').astype(float)
  #print(data_T)
  Lo_Ave=data_Lo.mean(skipna=True).astype(int)

  #-------------------------------------------------
  data_T=data_T['TotalTradedQuantity'].astype('string')
  for j in range(len(data_T)):
    data_T.iloc[j]=data_T.iloc[j].replace(",","")
  data_T=data_T.astype(int)
  data_T = data_T[:-1]
  #print(data_T)
  data_T1=data_T.mean(skipna=True).astype(int)
  data_VOL1.append(data_T1)
  Hi_Max1.append(Hi_Max)
  Hi_Ave1.append(Hi_Ave)
  Hi_Min1.append(Hi_Min)
  Lo_Max1.append(Lo_Max)
  Lo_Ave1.append(Lo_Ave)
  Lo_Min1.append(Lo_Min)
data_VOL['Ave_QNTY']= data_VOL1
data_VOL['Hi_Max']= Hi_Max1
data_VOL['Hi_Ave']= Hi_Ave1
data_VOL['Hi_Min']= Hi_Min1
data_VOL['Lo_Max']= Lo_Max1
data_VOL['Lo_Ave']= Lo_Ave1
data_VOL['Lo_Min']= Lo_Min1
#data_VOL
#list_comn[1]
#d=pd.read_excel("C:\\Users\\Admin\\Desktop\\Godwin\\Trade\\GapUp.xlsx")
#data_VOL=pd.DataFrame(d)
data_VOL.to_excel("GapUp.xlsx") # excel or csv
data_VOL.to_csv("GapUp.csv") # excel or csv

"""data_T=capital_market.price_volume_data(list_comn[1],period='1M').tail(14)
data_T=data_T['TotalTradedQuantity'].astype('string')
for i in range(len(data_T)):
  data_T.iloc[i]=data_T.iloc[i].replace(",","")
data_T=data_T.astype(int)
data_T.mean(skipna=True).astype(int)"""

'data_T=capital_market.price_volume_data(list_comn[1],period=\'1M\').tail(14)\ndata_T=data_T[\'TotalTradedQuantity\'].astype(\'string\')\nfor i in range(len(data_T)):\n  data_T.iloc[i]=data_T.iloc[i].replace(",","")\ndata_T=data_T.astype(int)\ndata_T.mean(skipna=True).astype(int)'

In [ ]:
#print(len(data_PC1))
#print(len(data2O))
data_VOL1=[]
Hi_Max1=[]
Hi_Min1=[]
Hi_Ave1=[]
Lo_Max1=[]
Lo_Min1=[]
Lo_Ave1=[]
for i in range(len(list_N)):
  data_T=capital_market.price_volume_data(list_N[i],period='1M').tail(14)
  #print(data_T)
  #-------------------------------------------------
  data_Tr=data_T[:-1]
  data_Tr= data_Tr.tail(7)
  Hi_Max=data_Tr['HighPrice'].max()
  Hi_Min=data_Tr['HighPrice'].min()

  data_Hi=data_Tr['HighPrice'].astype('string')
  for j in range(len(data_Hi)):
    data_Hi.iloc[j]=data_Hi.iloc[j].replace(",","")
  data_Hi=data_Hi.astype('string').astype(float)
  #print(data_T)
  Hi_Ave=data_Hi.mean(skipna=True).astype(int)


  Lo_Max=data_Tr['LowPrice'].max()
  Lo_Min=data_Tr['LowPrice'].min()

  data_Lo=data_Tr['LowPrice'].astype('string')
  for j in range(len(data_Lo)):
    data_Lo.iloc[j]=data_Lo.iloc[j].replace(",","")
  data_Lo=data_Lo.astype('string').astype(float)
  #print(data_T)
  Lo_Ave=data_Lo.mean(skipna=True).astype(int)

  #-------------------------------------------------
  data_T=data_T['TotalTradedQuantity'].astype('string')
  for j in range(len(data_T)):
    data_T.iloc[j]=data_T.iloc[j].replace(",","")
  data_T=data_T.astype(int)
  data_T = data_T[:-1]
  #print(data_T)
  data_T1=data_T.mean(skipna=True).astype(int)
  data_VOL1.append(data_T1)
  Hi_Max1.append(Hi_Max)
  Hi_Ave1.append(Hi_Ave)
  Hi_Min1.append(Hi_Min)
  Lo_Max1.append(Lo_Max)
  Lo_Ave1.append(Lo_Ave)
  Lo_Min1.append(Lo_Min)
data_VOL['Ave_QNTY']= data_VOL1
data_VOL['Hi_Max']= Hi_Max1
data_VOL['Hi_Ave']= Hi_Ave1
data_VOL['Hi_Min']= Hi_Min1
data_VOL['Lo_Max']= Lo_Max1
data_VOL['Lo_Ave']= Lo_Ave1
data_VOL['Lo_Min']= Lo_Min1
data_VOL
#list_comn[1]


In [45]:
data_Ta1=data_Ta

In [52]:
d=pd.read_excel("GapUp.xlsx")
data_VOL=pd.DataFrame(d)
data_S = pd.DataFrame()
data1=data.loc[1:,['SYMBOL','OPEN_PRICE','HIGH_PRICE','LOW_PRICE','CLOSE_PRICE','PREV_CLOSE']]
data_S=data_VOL.loc[1:,['GAP_UP','GAP_down']]
#print(data_S)
#data_VOL
#print(data_Ta[:-1])
data_Ta=data_Ta1
data_Ta=data_Ta[:-1]
da_Slope=data_Ta.tail(7)
print(da_Slope)
((da_Slope['HighPrice'].iloc[0]-da_Slope['HighPrice'].iloc[6])/da_Slope['HighPrice'].iloc[6])*6

   Symbol Series         Date  PrevClose  OpenPrice  HighPrice  LowPrice  \
13    NaN     EQ  06-Mar-2024     143.80     146.00     146.00    137.20   
14    NaN     EQ  07-Mar-2024     139.90     146.95     167.85    141.05   
15    NaN     EQ  11-Mar-2024     150.50     157.00     157.00    142.00   
16    NaN     EQ  12-Mar-2024     144.80     145.60     145.60    131.50   
17    NaN     EQ  13-Mar-2024     136.90     137.80     143.55    124.85   
18    NaN     EQ  14-Mar-2024     127.80     125.00     134.70    124.80   
19    NaN     EQ  15-Mar-2024     131.55     134.20     135.45    129.35   

    LastPrice  ClosePrice  AveragePrice TotalTradedQuantity Turnover  \
13      141.0      139.90        140.43            3,49,866      NaN   
14      150.0      150.50        156.98           10,42,372      NaN   
15      146.1      144.80        147.19            1,86,952      NaN   
16      138.4      136.90        137.80            3,94,602      NaN   
17      128.0      127.80      

0.46733111849390974